# 📊 Kruskal–Wallis Test in R (Jupyter Notebook)
This notebook runs through the Kruskal–Wallis test and post-hoc Dunn test, adapted from DataNovia’s "Kruskal‑Wallis test in R" tutorial.

In [ ]:
# Install required packages (if not already installed)
if (!requireNamespace("rstatix", quietly = TRUE)) install.packages("rstatix")
if (!requireNamespace("ggpubr", quietly = TRUE)) install.packages("ggpubr")
if (!requireNamespace("dunn.test", quietly = TRUE)) install.packages("dunn.test")
if (!requireNamespace("PMCMRplus", quietly = TRUE)) install.packages("PMCMRplus")

# Load libraries
library(rstatix)
library(ggpubr)
library(dplyr)
library(PMCMRplus)

In [ ]:
# Example dataset: three groups with different medians
set.seed(2025)
data <- data.frame(
  group = factor(rep(c("A", "B", "C"), each = 20)),
  value = c(rnorm(20, 5, 1), rnorm(20, 7, 1), rnorm(20, 9, 1))
)
head(data)

In [ ]:
# 🔍 Summary statistics by group (median & IQR)
data %>% group_by(group) %>% get_summary_stats(value, type = "median_iqr")

In [ ]:
# 📦 Visualize with boxplot + jitter
ggboxplot(data, x = "group", y = "value", add = c("jitter", "mean"),
          palette = "jco", title = "Values by Group")

In [ ]:
# 🧮 Kruskal–Wallis test
kw_test <- data %>% kruskal_test(value ~ group)
kw_test

In [ ]:
# 📉 Effect size for Kruskal–Wallis (eta-squared)
data %>% kruskal_effsize(value ~ group)

In [ ]:
# 🔍 Post‑hoc Dunn test with Bonferroni correction
dunn_res <- dunnTest(value ~ group, data = data, method = "bonferroni")
dunn_res

In [ ]:
# 📊 Visualize pairwise comparisons
dunn_df <- as.data.frame(dunn_res$res)
dunn_df <- dunn_df %>% mutate(p.sign = ifelse(P.adj < 0.05, "*", "ns"))

ggboxplot(data, x = "group", y = "value", palette = "jco") +
  stat_pvalue_manual(dunn_df, label = "p.sign", y.position = c(11, 12, 13)) +
  labs(title = "Post-hoc Dunn Test Results")

## ✅ Interpretation
- The **Kruskal–Wallis test** (`kruskal_test`) tells us *whether* there's at least one significant difference among groups.
- `kruskal_effsize()` outputs η², helping assess effect magnitude.
- The **Dunn test** pinpointed *which specific pairs* differ, with Bonferroni correction controlling for multiple testing.

---🎉 You're all set to run and interpret the Kruskal–Wallis test (with post‑hoc Dunn) in Jupyter using R!